Hello all, this is my first ever Jupiter Analyze and Kaggle notebook please dont be rude ;)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg' 
plt.style.use('fivethirtyeight')
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/crypto-currency-coint-market-cap-2017/coinmarketcap_06122017.csv")


Bitcoin and Cryptocurrencies: Full dataset, filtering, and reproducibility

In [ ]:
df.count()

In [ ]:
df.head()

Filtering all coints without capitalization

In [ ]:
market_cap=df
cap = market_cap.query('market_cap_usd >0')
print(cap.count())

Market capitalization for the top 10 coins 

In [ ]:
TOP_CAP_TITLE = 'Top 10 market capitalization'
TOP_CAP_YLABEL = '% of total cap'

top10=cap[:10].set_index('id')
top10 = top10.assign(market_cap_perc =
    lambda x: (x.market_cap_usd / cap.market_cap_usd.sum()) * 100)
barplot= top10.market_cap_perc.plot.bar(title=TOP_CAP_TITLE)
barplot.set_ylabel(TOP_CAP_YLABEL);



Making the plot easier to read

In [ ]:
COLORS = ['orange', 'green', 'orange', 'cyan', 'cyan', 'blue', 'silver', 'orange', 'red', 'green']
barplot1 =top10.market_cap_perc.plot.bar(title=TOP_CAP_TITLE,color=COLORS)

# Annotating the y axis with 'USD'
barplot1.set_ylabel("USD");
plt.yscale('log')
barplot1.set_xlabel("");

Volatility in cryptocurrencies

In [ ]:
volatility =df[["id","percent_change_24h","percent_change_7d"]]
volatility = volatility.set_index("id").dropna()
volatility = volatility.sort_values(by='percent_change_24h', ascending=True)

# Checking the first few rows
volatility.head()

Let's plot the top 10 biggest gainers and top 10 losers in market capitalization

In [ ]:

def top10_subplot(volatility_series, title):

    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 6))

    ax = volatility_series[:10].plot.bar(color="darkred", ax=axes[0])

    fig.suptitle(title)

    ax.set_ylabel('% change')

    ax = volatility_series[-10:].plot.bar(color="darkblue", ax=axes[1])
    
    return fig, ax

DTITLE = "24 hours top losers and winners"

# Calling the function above with the volatility.percent_change_24h series
# and title DTITLE 
fig, ax = top10_subplot(volatility.percent_change_24h, DTITLE)

Let's plot the top 10 biggest gainers and top 10 losers in market capitalization but let's focus on the week time

In [ ]:
volatility7d = volatility.sort_values(by='percent_change_7d', ascending=True)

WTITLE = "Weekly top losers and winners"

# Calling the top10_subplot function
fig, ax = top10_subplot(volatility7d.percent_change_7d, WTITLE)

Biggest Coints Capitalization (bigger then 10 billion)

In [ ]:
largecaps = market_cap.query('market_cap_usd >10000000000')

# Printing out largecaps
print(largecaps)

Check the market, how many do we have smaller coins and if there are more with micro capitalization.
We have used this values to set the coins groups:
* Big fish: >300 million
* Small cap: 50 million - 300 million
* Micro cap: Below 50 million

In [ ]:
def capcount(query_string):
    return cap.query(query_string).count().id

LABELS = ["bigfish", "small", "micro"]

bigfish = capcount("market_cap_usd > 3E+8")

small = capcount("market_cap_usd >= 5E+7 & market_cap_usd < 3E+8")

# ... and for nano
micro =  capcount("market_cap_usd < 5E+7")

# Making a list with the 3 counts
values = [bigfish, small, micro]

# Plotting them with matplotlib 
plt.bar(range(len(values)), values, tick_label=LABELS);